In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [2]:
page = requests.get('https://tvn24.pl/polska')

soup = BeautifulSoup(page.content, 'html.parser')
page_content = soup.find_all('article',class_='teaser-wrapper__content default-teaser default-teaser--standard')

In [11]:
ignored_paragraph_starts = ['OGLĄDAJ TELEWIZJĘ NA ŻYWO', ]

In [12]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False

In [17]:
def scrap_article_text_tvn24(link):
    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find_all('p', class_='paragraph')

    text = ''
    for paragraph in paragraphs:
        paragraph_text = paragraph.get_text().strip()
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    time_string = soup.find('time', class_='article-top-bar__date').get_text().strip()
    when_published = PolishLocaleHelper.parse_pl_date_tvn24(time_string)

    title_int = soup.find('h1', class_='article-top-bar__title').get_text().strip()
    lead_text_int = soup.find('p', class_='lead-text').get_text().strip()

    author = soup.find_all('dd', class_='article-top-bar__meta-item')[0].get_text().strip()
    source = soup.find_all('dd', class_='article-top-bar__meta-item')[1].get_text().strip()

    return text, when_published, title_int, lead_text_int, author, source

In [22]:
result_list = list()
for article in page_content:
    # print(article)
    title = article.find('h2',class_='heading article-title').get_text().strip()
    lead_text = article.find('p',class_='article-lead__text').get_text().strip()
    link = article.find_all('a', href=True)[0]['href']
    # print(f'link: {link}')
    webart = WebArticle(title, lead_text, link, *scrap_article_text_tvn24(link))
    # print(f'webarticle scrapped: {webart}')
    result_list.append(webart)
    print(f'webart.title: {webart.title}')
    print(f'webart.lead_text: {webart.lead_text}')
    print(f'webart.link: {webart.link}')
    print(f'webart.text: {webart.text}')
    print(f'webart.when_published: {webart.when_published}')
    print(f'webart.title_int: {webart.title_int}')
    print(f'webart.lead_text_int: {webart.lead_text_int}')
    print(f'webart.author: {webart.author}')
    print(f'webart.source: {webart.source}')

webart.title: "Uprowadzony Polak wrócił już do kraju i jest w dobrym stanie"
webart.lead_text: Polski marynarz został uprowadzony w grudniu zeszłego roku w Zatoce Gwinejskiej.
webart.link: https://tvn24.pl/swiat/polski-marynarz-uwolniony-po-uprowadzeniu-w-zatoce-gwinejskiej-5562224
webart.text: ('Jak przekazało MSZ w komunikacie, "członkowie załogi statku porwanego w połowie grudnia 2021 r. u wybrzeży Nigerii, w tym obywatel RP, zostali uwolnieni". "Uprowadzony Polak wrócił już do kraju i jest w dobrym stanie" - poinformowano. "Marynarze zostali uwolnieni dzięki działaniom armatora oraz sprawnej, profesjonalnej współpracy służby konsularnej i innych polskich instytucji z władzami Danii i Nigerii oraz z firmą zatrudniającą załogę statku" - napisano w komunikacie resortu. OGLĄDAJ TVN24 W INTERNECIE W TVN24 GO Po stronie polskiej sprawą zajmował się międzyresortowy zespół pod przewodnictwem wiceministra Piotra Wawrzyka. Wydarzenie z połowy grudnia zeszłego roku było szóstym porwaniem obyw

In [20]:
link1 = 'https://tvn24.pl/polska/lublin-zatrzymali-falszywego-policjanta-ktory-odbieral-pieniadze-od-oszukanych-osob-5561887'
scrap_article_text_tvn24(link1)

('- Scenariusz był we wszystkich przypadkach identyczny. Na telefon starszych osób dzwonił mężczyzna podający się za funkcjonariusza i przekonywał, że ich pieniądze są zagrożone. Pod tą legendą zmuszał ich do wypłaty pieniędzy z placówek bankowych i przekazania bezpośrednio innemu rzekomemu mundurowemu. W ten sposób oszukani zostali mieszkańcy województw: lubelskiego, podkarpackiego oraz lubuskiego - mówi podinspektor Radosław Dawidek z zespołu prasowego lubelskiej policji. Kryminalni z Komendy Wojewódzkiej Policji w Lublinie oraz mundurowi z jednostki w Lubartowie wpadli na trop 25-letniego mieszkańca województwa podlaskiego, który - zgodnie z ich ustaleniami - odbierał od oszukanych osób pieniądze. Podając się przy tym za policjanta. Z ustaleń śledczych wynika, że w grudniu i styczniu 25-latek odebrał od czterech pokrzywdzonych łącznie ponad 160 tysięcy złotych. Zabrał też pieniądze w obcej walucie. Piątą ofiarą miał być mieszkaniec powiatu lubartowskiego, jednak ostatecznie nie udał